Como primeira etapa, você precisa instalar a biblioteca Beautiful Soup usando seu terminal ou laboratório jupyter.


In [21]:
!pip3 install beautifulsoup4


Importando bibliotecas necessárias


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [23]:
no_pages = 2

def get_data(pageNo):  
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    for d in soup.findAll('div', attrs={'class':'a-section a-spacing-none aok-relative'}):
        #print(d)
        name = d.find('span', attrs={'class':'zg-text-center-align'})
        n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('a', attrs={'class':'a-size-small a-link-child'})
        rating = d.find('span', attrs={'class':'a-icon-alt'})
        users_rated = d.find('a', attrs={'class':'a-size-small a-link-normal'})
        price = d.find('span', attrs={'class':'p13n-sc-price'})

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(n[0]['alt'])
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('-1')

        if users_rated is not None:
            #print(price.text)
            all1.append(users_rated.text)
        else:
            all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [24]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Book Name','Author','Rating','Customers_Rated', 'Price'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [25]:
df = pd.read_csv("amazon_products.csv")
df.shape


(100, 5)

In [26]:
df.head(61)

,Book Name,Author,Rating,Customers_Rated,Price
0,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5 out of 5 stars,"27,794",₹399.00
1,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6 out of 5 stars,"17,131",₹339.00
2,The Psychology of Money,Morgan Housel,4.6 out of 5 stars,"12,104",₹295.00
3,My First Book of Pencil Control : Practice Pat...,Wonder House Books,4.4 out of 5 stars,"7,011",₹89.00
4,25 Ki Series - 25 NDA Maths Mock Test Series,Arpit Choudhary,4.5 out of 5 stars,17,0
...,...,...,...,...,...
56,My Activity- ABC Colouring Book,Dreamland Publications,4.4 out of 5 stars,"2,000",₹78.00
57,My Big Book of Creative Crayons,Wonder House Books,4.4 out of 5 stars,206,₹149.00
58,"Environment - 8/edition, 2021",Shankar,3.8 out of 5 stars,38,₹204.00
59,The Power of your Subconscious Mind,Joseph Murphy,4.5 out of 5 stars,"34,670",₹149.00


In [27]:
df['Rating'] = df['Rating'].apply(lambda x: x.split()[0])
df['Rating'] = pd.to_numeric(df['Rating'])
df["Price"] = df["Price"].str.replace('₹', '')
df["Price"] = df["Price"].str.replace(',', '')
df['Price'] = df['Price'].apply(lambda x: x.split('.')[0])
df['Price'] = df['Price'].astype(int)
df["Customers_Rated"] = df["Customers_Rated"].str.replace(',', '')
df['Customers_Rated'] = pd.to_numeric(df['Customers_Rated'], errors='ignore')
df.head()


,Book Name,Author,Rating,Customers_Rated,Price
0,My First Library: Boxset of 10 Board Books for...,Wonder House Books,4.5,27794,399
1,Ikigai: The Japanese secret to a long and happ...,Héctor García,4.6,17131,339
2,The Psychology of Money,Morgan Housel,4.6,12104,295
3,My First Book of Pencil Control : Practice Pat...,Wonder House Books,4.4,7011,89
4,25 Ki Series - 25 NDA Maths Mock Test Series,Arpit Choudhary,4.5,17,0


In [28]:
df.dtypes


Book Name           object
Author              object
Rating             float64
Customers_Rated      int64
Price                int64
dtype: object

In [29]:
df.replace(str(0), np.nan, inplace=True)
df.replace(0, np.nan, inplace=True)

In [30]:
count_nan = len(df) - df.count()


In [31]:
count_nan


Book Name          0
Author             0
Rating             0
Customers_Rated    1
Price              1
dtype: int64

In [32]:
df = df.dropna()


Vamos descobrir quais autores tiveram o livro mais caro. Você irá visualizar os resultados para esses 20 autores principais.



In [37]:
data = df.sort_values(["Price"], axis=0, ascending=False)[:15]
data

,Book Name,Author,Rating,Customers_Rated,Price
62,How to Prepare for QUANTITATIVE APTITUDE for C...,Arun Sharma,4.7,29.0,808.0
51,My First Complete Learning Library: Boxset of ...,Wonder House Books,4.6,5340.0,790.0
66,"NCERT textbooks physics, chemistry and biology...",NCERT,4.1,1022.0,650.0
17,Indian Polity - For Civil Services and Other S...,M. Laxmikanth,4.6,11029.0,615.0
90,Mathematics for Class 11 (set of 2 volumes) Ex...,R.D. Sharma,4.6,1091.0,600.0
41,A Modern Approach to Verbal & Non-Verbal Reaso...,R.S. Aggarwal,4.4,6070.0,544.0
86,Indian Art and Culture for Civil Services and ...,Nitin Singhania,4.6,3313.0,540.0
15,Objective NCERT at your FINGERTIPS for NEET-AI...,MTG Editorial Board,4.6,2335.0,539.0
35,The Intelligent Investor (English) Paperback –...,Benjamin Graham,4.5,25466.0,518.0
96,Objective NCERT at your Fingertips for NEET-AI...,MTG Editorial Board,4.5,1899.0,510.0
